In [6]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

## Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
## Training Parameters 
learning_rate = 0.001
num_iter = 500
batch_size = 128
display_step = 10

## Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.80 # Dropout, probability to keep units

## tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [3]:
## Defining wrappers for convolution functions
def conv2D(x,W,b,stride = 1):
    x = tf.nn.conv2d(x, W, strides = [1,stride,stride,1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def max_pool(x,k=2,s=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1],strides=[1,s,s,1], padding='SAME')
    
    
## Creating convolution model
def conv_net(x, weights, biases, dropout):
    ## reshaping 1D vectors of 784 dimentions to image shape [28*28*1]
    ## tensor input becomes 4D [Batch_size, height, width, channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    ## First convolution layer
    conv1 = conv2D(x, weights['W1'], biases['b1'], stride = 1)
    conv1 = max_pool(conv1, k=2, s=2)
    
    ## Second convolution layer
    conv2 = conv2D(conv1, weights['W2'], biases['b2'], stride = 1)
    conv2 = max_pool(conv2, k=2, s=2)
    
    ## Third convolution layer
#     conv3 = conv2D(conv2, weights['W3'], biases['b3'], stride = 1)
#     conv3 = max_pool(conv3, k=2, s=1) 
    
    ## Fully connected layer
    ## Reshaping the output from 3D image shape to 1D vectors for fully connected layers
    fc1 = tf.reshape(conv2, [-1, weights['fw1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['fw1']), biases['fb1'])
    fc1 = tf.nn.relu(fc1)

    ## Applying Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    ## Output class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [4]:
tf.set_random_seed(42)
## Initializing layer weights & biases
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'W1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'W2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
#     3x3 conv, 64 inputs, 64 outputs
#     'W3': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'fw1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([32])),
    'b2': tf.Variable(tf.random_normal([64])),
#     'b3': tf.Variable(tf.random_normal([64])),
    'fb1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

## Obtaining predictions using softmax layer
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

## Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


## Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
## Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('loss',loss_op)
merged = tf.summary.merge_all()
    
writer = tf.summary.FileWriter('./MNIST', tf.get_default_graph())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [5]:
## Start training
with tf.Session() as sess:

 
    ## Run the initializer
    sess.run(init)
    
    for step in range(1, num_iter+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        ## Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            ## Calculate batch loss and accuracy
            loss, acc,summary = sess.run([loss_op, accuracy,merged], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            writer.add_summary(summary,step)
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
 
            ##saving checkpoints after every 10 iterations
            save_path = tf.train.Saver().save(sess, './MNIST/chkpts.ckpt', global_step=step)
            
    
    ## Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))
writer.close()

Step 1, Minibatch Loss= 57729.7773, Training Accuracy= 0.102
Step 10, Minibatch Loss= 20491.8008, Training Accuracy= 0.320
Step 20, Minibatch Loss= 8457.9629, Training Accuracy= 0.555
Step 30, Minibatch Loss= 7050.4365, Training Accuracy= 0.641
Step 40, Minibatch Loss= 5656.5430, Training Accuracy= 0.695
Step 50, Minibatch Loss= 3850.0415, Training Accuracy= 0.805
Step 60, Minibatch Loss= 2168.4045, Training Accuracy= 0.820
Step 70, Minibatch Loss= 1045.3065, Training Accuracy= 0.898
Step 80, Minibatch Loss= 1720.8219, Training Accuracy= 0.891
Step 90, Minibatch Loss= 1155.6749, Training Accuracy= 0.914
Step 100, Minibatch Loss= 1675.8074, Training Accuracy= 0.891
Step 110, Minibatch Loss= 2080.1138, Training Accuracy= 0.891
Step 120, Minibatch Loss= 729.0461, Training Accuracy= 0.914
Step 130, Minibatch Loss= 1316.6907, Training Accuracy= 0.875
Step 140, Minibatch Loss= 2814.5793, Training Accuracy= 0.883
Step 150, Minibatch Loss= 1305.0361, Training Accuracy= 0.945
Step 160, Minibatc

In [ ]:
tf.reset_default_graph()